In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma

plt.style.use('dark_background')

In [2]:
from helper_functions import load_vt_unfiltered_df, load_48_hd_files, clean_48_zip_files

In [3]:
VT_FILE_PATH = 'data_files/VT_unfiltered_Feb.txt'
HD_48_FILE_PATH = 'data_files/unzipped_files/'

# only run this once!
# clean_48_zip_files(HD_48_FILE_PATH)

In [4]:
df_vt, vt_first_row = load_vt_unfiltered_df(VT_FILE_PATH, True, 'sum')

KeyboardInterrupt: 

In [ ]:
df_48, df_48_first_row = load_48_hd_files(HD_48_FILE_PATH, True, True)


In [ ]:
df_vt

In [ ]:
df_48

In [ ]:
intersecting_nucleotides = df_vt.index.intersection(df_48.index)

In [ ]:
# Creates new dfs which only contain the intersecting nuc sequences so they can be compared
# The order of indices is the same in both dfs so they can be easily compared!

df_vt_itx = df_vt.loc[intersecting_nucleotides]
df_48_itx = df_48.loc[intersecting_nucleotides]

In [56]:
def custom_diff_metric(df_48,df_vt):
    """ Creates a custom correlation based on masked values within given arrays"""
    # Use for df_vt and df_48 comparision

    a_masked = ma.masked_values(df_48, 0)
    b_masked = ma.masked_values(df_vt, 0)

    # this ensures that the mask is only applied when both columns are 0 for a given seq
    new_mask = np.logical_and(a_masked.mask, b_masked.mask)

    # update masks:
    a_masked.mask = new_mask
    b_masked.mask = new_mask

    c = ma.absolute(ma.subtract(a_masked, b_masked))
    d = ma.divide(c, b_masked)
    d = ma.mean(d)*100
    
    matched_rows = len(a_masked.nonzero())

    return d, c.count()
    
def equal_cols(col_48_1, col_vt_1):
    # Count equals in two columns. Return a tuple with: (count true, count false, count true/total)
    bool_count = np.equal(col_48_1, col_vt_1)
    positive_count = np.sum(bool_count)
    negative_count = len(bool_count) - positive_count
    positive_proportion = positive_count / len(bool_count)

    return positive_count, negative_count, positive_proportion

def equal_cols_masked(col_48_1, col_vt_1):
    a_masked = ma.masked_values(col_48_1, 0)
    b_masked = ma.masked_values(col_vt_1, 0)

    # this ensures that the mask is only applied when both columns are 0 for a given seq
    new_mask = np.logical_and(a_masked.mask, b_masked.mask)

    # update masks:
    a_masked.mask = new_mask
    b_masked.mask = new_mask

    # bool_count = ma.equal(a_masked, b_masked)
    bool_count = np.isclose(a_masked, b_masked, rtol=0.2, atol=0)
    positive_count = bool_count.sum()
    negative_count = bool_count.count() - positive_count
    positive_proportion = positive_count / bool_count.count()

    return positive_count, negative_count, positive_proportion

In [16]:
# EQUALS Columns Approach

col_vt = list(df_vt_itx.columns)
col_48 = list(df_48_itx.columns)

corr_dict = {}

for col_1 in col_vt:
    for col_2 in col_48:
        corr_dict[(col_1,col_2)] = equal_cols(df_48_itx[col_2],df_vt_itx[col_1])
        
corr_dict_keys = []
corr_tuple = []

for key, val in corr_dict.items():
    corr_dict_keys.append(key)
    corr_tuple.append(val)

corr_equals_df = pd.DataFrame(corr_tuple,
                            index=corr_dict_keys, columns=['Positive', 'Negative', 'Proportion'])


In [20]:
corr_equals_df.sort_values(by='Proportion', ascending=False).to_csv('equals_correlation.csv')

In [31]:
# Masked EQUALS Columns Approach
col_vt = list(df_vt_itx.columns)
col_48 = list(df_48_itx.columns)

corr_dict = {}

for col_1 in col_vt:
    for col_2 in col_48:
        corr_dict[(col_1,col_2)] = equal_cols_masked(df_48_itx[col_2],df_vt_itx[col_1])
        
corr_dict_keys = []
corr_tuple = []

for key, val in corr_dict.items():
    corr_dict_keys.append(key)
    corr_tuple.append(val)

corr_equals_masked_df = pd.DataFrame(corr_tuple,
                            index=corr_dict_keys, columns=['Positive', 'Negative', 'Proportion'])

In [32]:
corr_equals_masked_df.sort_values(by='Positive', ascending=False)

,Positive,Negative,Proportion
"(VT20, 20180222-57NYooVH-VT__R7F2_RN1RP3)",29136,153523,0.159510
"(VT23, 20180222-57NYooVH-VT__R7F2_RN1RP3)",28908,143882,0.167301
"(VT19, 20180222-57NYooVH-VT__R7F2_RN1RP3)",28858,148776,0.162458
"(VT23, 20180222-70OOooVH-VT__R7F4_RN1RP3)",28838,142644,0.168169
"(VT24, 20180222-57NYooVH-VT__R7F2_RN1RP3)",28833,144388,0.166452
...,...,...,...
"(VT15, 20180222-24NYsaVH-VG__R3F17_RN1RP3)",0,1534,0.000000
"(VT15, 20180222-24OOooVH-VG__R9F9_RN1RP1)",0,352,0.000000
"(VT15, 20180222-24OOooVH-VG__R9F10_RN1RP2)",0,309,0.000000
"(VT15, 20180222-24OOooVH-VG__R9F11_RN1RP3)",0,356,0.000000


In [57]:
# Custom Difference Metric
col_vt = list(df_vt_itx.columns)
col_48 = list(df_48_itx.columns)

corr_dict = {}

for col_1 in col_vt:
    for col_2 in col_48:
        corr_dict[(col_1,col_2)] = custom_diff_metric(df_48_itx[col_2],df_vt_itx[col_1])
        
corr_dict_keys = []
corr_tuple = []

for key, val in corr_dict.items():
    corr_dict_keys.append(key)
    corr_tuple.append(val)

custom_diff_df = pd.DataFrame(corr_tuple,
                            index=corr_dict_keys, columns=['Custom_Difference', 'Matched_Rows'])

In [60]:
custom_diff_df.sort_values(by='Custom_Difference', ascending=True).to_csv('custom_diff_metric_comparison.csv')

## Compare Totals Approach:

1. Load untruncated sequences.
2. Sum up across columns. 
3. Save and compare csv files manually.

In [63]:
df_vt, vt_first_row = load_vt_unfiltered_df(VT_FILE_PATH, trim=False, keep='sum')

Length of DF_VT before: 3095530
Len after 3095526


In [62]:
df_48, df_48_first_row = load_48_hd_files(HD_48_FILE_PATH, False, False)

Length of DF data_files/unzipped_files\20180222-24NYooVH-VG.txt before: 3048
0
Len after 3048
Length of DF data_files/unzipped_files\20180222-24NYsaVH-VG.txt before: 438900
0
Len after 438900
Length of DF data_files/unzipped_files\20180222-24OOooVH-VG.txt before: 2971
0
Len after 2971
Length of DF data_files/unzipped_files\20180222-24OOsaVH-VG.txt before: 384306
0
Len after 384306
Length of DF data_files/unzipped_files\20180222-24WIooVH-VG.txt before: 4831
0
Len after 4831
Length of DF data_files/unzipped_files\20180222-24WIooVL-VG.txt before: 3564
0
Len after 3564
Length of DF data_files/unzipped_files\20180222-24WIsaVH-VG.txt before: 4636
0
Len after 4636
Length of DF data_files/unzipped_files\20180222-24WIsaVL-VG.txt before: 231618
2
Len after 231616
Length of DF data_files/unzipped_files\20180222-57NYooVH-VT.txt before: 219326
0
Len after 219326
Length of DF data_files/unzipped_files\20180222-57NYsaVH-VT.txt before: 219520
0
Len after 219520
Length of DF data_files/unzipped_files\2

In [66]:
df_vt.sum(axis=0).to_csv('sum_count_vt_comparison.csv')

In [68]:
df_48.sum(axis=0).to_csv('sum_count_48_comparision.csv')

# Untrimmed Appraoach

Do the same comparisions but without any truncations to the nucleotide sequences.

In [69]:
print(f'Len 48 DF: {len(df_48)}\nLen VT DF: {len(df_vt)}')

Len 48 DF: 2239967
Len VT DF: 3095526


In [70]:
intersecting_nucleotides = df_vt.index.intersection(df_48.index)
# Creates new dfs which only contain the intersecting nuc sequences so they can be compared
# The order of indices is the same in both dfs so they can be easily compared!

df_vt_itx = df_vt.loc[intersecting_nucleotides]
df_48_itx = df_48.loc[intersecting_nucleotides]

In [71]:
intersecting_nucleotides

Index(['AAAAAAATATTATTCGCAATTCCTTTAGTGGTACCTTTCTATTCTCAC',
       'AAAAAAATGCTTTTCGCTATTCCACTTGTGGTACCTTTCTATTCTCACTCTCAGTTTACGTAGCTGCATCAGGGTGGAGGT',
       'AAAAAACAGCTGTTTGCGATCCCTCTAGTGGTACCTTTCTATTCTCACTCTCAGTTTACGTAGCTGCATCAGGGTGGAGGT',
       'AAAAAACTACTATTCGCGATTCCGCTTGTGGTACCTTTCTATTCTCACTCTTCTATCTGTATCTACCCGTGTGGTGGAGGT',
       'AAAAAACTACTATTCGCGATTCCGCTTGTGGTACCTTTCTATTCTCACTCTTCTATGTGTGCTTACTTCTGTGGTGGAGGT',
       'AAAAAACTACTATTCGCGATTCCGCTTGTGGTACCTTTCTATTCTCACTCTTCTCATTGTTACGTTTACTGTGGTGGAGGT',
       'AAAAAACTACTATTCGCGATTCCGCTTGTGGTACCTTTCTATTCTCACTCTTCTCCGTGTATCCTGGAATGTGGTGGAGGT',
       'AAAAAACTACTATTCGCGATTCCGCTTGTGGTACCTTTCTATTCTCACTCTTCTCCGTGTATGTTCGACTGTGGTGGAGGT',
       'AAAAAACTACTATTCGCGATTCCGCTTGTGGTACCTTTCTATTCTCACTCTTCTGAATGTGGTTCTATCTGTGGTGGAGGT',
       'AAAAAACTACTATTCGCGATTCCGCTTGTGGTACCTTTCTATTCTCACTCTTCTGCTTGTAAACATGGTTGTGGTGGAGGT',
       ...
       'AAAGAATTATTATTCGCAATTCCTTTAGTGGTACCTTTCTATTCTCAC',
       'AAATAATTATTATTCGCAATTCCTTTAGTGGTACC

In [73]:
len(intersecting_nucleotides)/len(df_vt)*100

1.5601225769061542

In [74]:
# Given the low intersection, might as well not continue on this. 